In [1]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.nb.stop_words import STOP_WORDS
# importer norskspråkelig modul i spacy
import nb_core_news_sm

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

nrk = pd.read_csv('include/nrk.csv', sep='\t', skiprows=0, header=0)
newest_doc = nrk['content']

nlp = spacy.load("nb_core_news_sm")

# My list of stop words.
stop_list = ["sier"]

# Updates spaCy's default stop words list with my additional words.
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True


def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them.
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)


def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc


# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer, name='lemmatizer', after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

doc_list = []
# Iterates through each article in the corpus.
for doc in tqdm(newest_doc):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)
    doc_list.append(pr)

# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=10,
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

pprint(lda_model.print_topics(num_words=10))


/home/osboxes/.local/lib/python3.7/site-packages/ipykernel_launcher.py:56: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



[(0,
  '0.087*"skrive" + 0.045*"handling" + 0.045*"mann" + 0.045*"Helsingborg" + '
  '0.045*"Helsingborgs" + 0.045*"avis" + 0.045*"formening" + 0.045*"mistenkt" '
  '+ 0.045*"politi" + 0.045*"pågripe"'),
 (1,
  '0.019*"skrive" + 0.019*"ferdig" + 0.019*"valg" + 0.019*"ringpingvine" + '
  '0.019*"skje" + 0.019*"forrige" + 0.019*"Sinn" + 0.019*"160" + '
  '0.019*"sprengladning" + 0.019*"pågripelse"'),
 (2,
  '0.019*"skrive" + 0.019*"Sinn" + 0.019*"ringpingvine" + 0.019*"forrige" + '
  '0.019*"Féin" + 0.019*"planlegge" + 0.019*"Reuters" + 0.019*"sprengladning" '
  '+ 0.019*"160" + 0.019*"uke"'),
 (3,
  '0.080*"Féin" + 0.080*"valg" + 0.080*"Sinn" + 0.042*"parti" + 0.042*"helgen" '
  '+ 0.042*"natt" + 0.042*"Irland" + 0.042*"enstreorientert" + 0.042*"ferdig" '
  '+ 0.042*"160"'),
 (4,
  '0.091*"ringpingvine" + 0.047*"koloni" + 0.047*"blive" + 0.047*"bestand" + '
  '0.047*"nyhetsbyrå" + 0.047*"skrive" + 0.047*"sist" + 0.047*"1970" + '
  '0.047*"Antarktisk" + 0.047*"tall"'),
 (5,
  '0.019*"sk